In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [22]:
def read_results(root_dir, experiment_name):
    score_results = pd.read_csv(os.path.join(root_dir, "results", f"{experiment_name}_results.csv"), index_col=None)
    for column in score_results.columns:
        if "score:" in column:
            score_name = column.split(":")[1]
            break
    return score_results, score_name

def table_mean_std(score_results, score, prob_types=None, dataset=None):
    score_results = score_results.sort_values(["model", "dataset", "eval_split", "n_shots", "output_prob_type"])
    score_results = score_results.groupby(["model", "dataset", "eval_split", "n_shots", "output_prob_type"]).agg({
        f"score:{score}": ["mean", "std"], 
    })
    if prob_types is not None:
        prob_types = [pt for pt in prob_types if pt in score_results.index.get_level_values("output_prob_type").unique()]
        score_results = score_results.loc[(slice(None), slice(None), slice(None), slice(None), prob_types), :].sort_index(level=["model", "dataset", "eval_split", "n_shots"])
    if dataset is not None:
        dataset = [ds for ds in dataset if ds in score_results.index.get_level_values("dataset").unique()]
        score_results = score_results.loc[(slice(None), dataset, slice(None), slice(None), slice(None)), :].sort_index(level=["model", "dataset", "eval_split", "n_shots"])
    return score_results

root_dir = "./"
experiment = "gpt2-xl_n100"
# experiment = "cb"
# experiment = "gpt3-text-davinci-003_n100"
# prob_types = ["original_probs", "content_free_['idk']"]
prob_types = None
# dataset = ["cb", "trec"]
dataset = ["cb", "rte", "sst2", "trec", "agnews"]
score_results, score = read_results(root_dir, experiment)
table_mean_std(score_results, score, prob_types=prob_types, dataset=dataset)#.index.get_level_values("dataset").unique()

score:accuracy   
                                                                                mean   
model   dataset eval_split n_shots output_prob_type                                    
gpt2-xl agnews  test       0       content_free_['[MASK]', 'N/A', '']       0.689868  \
                                   content_free_['idk']                     0.642368   
                                   original_probs                           0.447105   
                                   probs_rescaled_train_queries             0.663263   
        cb      test       0       content_free_['[MASK]', 'N/A', '']       0.089286   
...                                                                              ...   
        trec    test       4       probs_rescaled_train_queries             0.469200   
                           8       content_free_['[MASK]', 'N/A', '']       0.466400   
                                   content_free_['idk']                     0.454000   
                                   original_probs                           0.269600   
                                   probs_rescaled_train_queries             0.490800   

                                                                                 
                                                                            std  
model   dataset eval_split n_shots output_prob_type                              
gpt2-xl agnews  test       0       content_free_['[MASK]', 'N/A', '']  0.000000  
                                   content_free_['idk']                0.000000  
                                   original_probs                      0.000000  
                                   probs_rescaled_train_queries        0.011964  
        cb      test       0       content_free_['[MASK]', 'N/A', '']  0.000000  
...                                                                         ...  
        trec    test       4       probs_rescaled_train_queries        0.032019  
                           8       content_free_['[MASK]', 'N/A', '']  0.042998  
                                   content_free_['idk']                0.052154  
                                   original_probs                      0.040924  
                                   probs_rescaled_train_queries        0.049165  

[68 rows x 2 columns]

In [20]:
from src.data import ClassificationDataset

datasets = ClassificationDataset.DATASETS
for dataset_name in datasets:
    dataset = ClassificationDataset(
        "./",
        None,
        dataset_name,
        n_shot=0,
        random_state=None
    )
    print(dataset_name)
    p_train = np.bincount(dataset._data["train_labels"], minlength=len(dataset.label_dict))
    p_train = p_train / p_train.sum()
    print(p_train)
    p_test = np.bincount(dataset._data["test_labels"], minlength=len(dataset.label_dict))
    p_test = p_test / p_test.sum()
    print(p_test)
    print()

agnews
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]

trec
[0.16434336 0.15315481 0.22432135 0.2131328  0.22927366 0.01577403]
[0.226 0.162 0.13  0.276 0.188 0.018]

cb
[0.476 0.064 0.46 ]
[0.5        0.08928571 0.41071429]

rte
[0.49839357 0.50160643]
[0.47292419 0.52707581]

sst2
[0.4783237 0.5216763]
[0.50082372 0.49917628]

dbpedia
[0.07108142 0.07066141 0.0699414  0.07116142 0.07210144 0.07124142
 0.07214144 0.06960139 0.07030141 0.07090142 0.07296146 0.07314146
 0.07188144 0.07288146]
[0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
 0.07142857 0.07142857]

